In [8]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys
import os

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'name': 'test',
 'products':     description       stock_code
 0   Brazil nuts           363477
 1  GB chocolate           260803
 2          fake  fake_stock_code}

In [3]:
# Get and parse a product

code = w['products']['stock_code'].iat[-1]
response = cd.get_product(code)
cd.parse_product(response)

OrderedDict([('stock_code', 'fake_stock_code'),
             ('name', None),
             ('description', None),
             ('size', None),
             ('sale_price', None),
             ('price', None),
             ('discount_percentage', None),
             ('unit_price', None),
             ('datetime', '2017-11-20T17:50:04')])

In [4]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=False)
f

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 1.28 s


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.854739,$3.80/100G,2017-11-20 17:50:04
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,3.5,3.89,10.025707,$3.50/100G,2017-11-20 17:50:04
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-11-20 17:50:05


In [5]:
# Get many products asynchronously and parse

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=True)
f

CPU times: user 260 ms, sys: 4 ms, total: 264 ms
Wall time: 711 ms


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,3.5,3.89,10.025707,$3.50/100G,2017-11-20 17:50:05
1,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.854739,$3.80/100G,2017-11-20 17:50:05
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-11-20 17:50:06


In [6]:
# Filter sales

g = cd.filter_sales(f)
g

,name,sale_price,price,discount_percentage
0,Green & Blacks Chocolate Block Organic Dark Ch...,3.5,3.89,10.025707
1,Ceres Organics Brazil Nuts 250g,9.5,11.29,15.854739


In [9]:
# Email sales

domain = os.environ['MAILGUN_DOMAIN'] # replace with your Mailgun domain
key = os.environ['MAILGUN_KEY'] # replace with your Mailgun API key
cd.email(g, ['brainbummer@mailinator.com'], domain, key)

<Response [200]>

In [10]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.yaml')

,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.854739,$3.80/100G,2017-11-20 17:50:24
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,3.5,3.89,10.025707,$3.50/100G,2017-11-20 17:50:24
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-11-20 17:50:25


In [11]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', OUT_DIR/'products.csv', domain, key)
%less {OUT_DIR/'products.csv'}